#### LINQ

LINQ significa Language INtegrated Query y es un conjunto de tecnologias basadas en la integracion directamente dentro del lenguaje C# de la capacidad de escribir consultas hacia diferentes fuentes de datos.

Con LINQ una consulta es una construccion de "primera clase" con type-checking en tiempo de compilacion, intellisense y una estructura que nos remite a la sintaxis de SQL. Los origenes de datos pueden ser tan diversos como un simple array, colecciones de objetos de cualquier tipo, documentos XML o tablas en una base de datos.

Cuando se escribe una consulta la parte mas visible de LINQ que esta "integrada al lenguaje" es la **query expression**. Estas expresiones son escritas de manera declarativa en lo que se conoce como **query syntax**, como veremos en el siguiente ejemplo simple donde filtramos un array de numeros para obtener los pares y multiplicarlos por 10:

In [ ]:
int[] numeros = Enumerable.Range(-10, 30).ToArray();

var pares_x_10 = from num in numeros
                    where num % 2 == 0
                    select num * 10;

foreach (var n in pares_x_10) Console.Write($"{n} ");

Algunas caracteristicas importantes de estas query expressions:

- Una query no es ejecutada hasta que se itera (foreach) sobre la variable que usamos para crear la consulta. En nuestro caso seria hasta no hacer el foreach sobre *pares_x_10*
- Podemos expresar las consultas tambien usando la sintaxis de metodos (fluent), no hay diferencia de performance entre ambas
- Algunas operaciones como Count o Max no pueden expresarse en query syntax y deberian ser realizadas con llamadas a metodos
- Las expresiones de query pueden ser compiladas como delegados (colecciones en memoria) o como arboles de expresiones, usando clases en System.Linq.Expressions (colecciones en base de datos)
- Las interfaces involucradas son IEnumerable<T> en el primer caso e IQueryable<T> en el segundo

En mi opinion y gusto personal, la fluent syntax (o method syntax) es mas clara para entender y si bien perdemos la "integracion" del lenguaje, creo que se expresa mejor lo que se esta haciendo, los pasos puntuales de cada etapa de la consulta.

Como adicionales, primero en la sintaxis fluent no necesitamos la clausula select final, salvo que estemos proyectando parte del objeto base o lo estemos modificando. Segundo, las operaciones que no disponemos en la query syntax son mas naturales incluirlas en la method syntax, sin mezclar ambas que resultaria confuso.

In [ ]:
int[] numeros = Enumerable.Range(-10, 30).ToArray();

//  para obtener la cantidad de elementos con query syntax
//
var cantidad = (from num in numeros
                    where num % 2 == 0
                    select num * 10).Count();

Console.Write($"Query: {cantidad} ");

//  lo mismo en fluent
//
cantidad = numeros
              .Where(n => n % 2 == 0)
              .Select(x => x * 10)
              .Count();

Console.Write($"Fluent: {cantidad} ");


Cada uno elegira la que mas le guste!

Vamos con un par de ejemplos mas, como aparece en el slide de la presentacion: tenemos 5 nombres, primero seleccionamos aquellos que contienen una "a" luego los ordenamos de manera ascendente (el default) respecto a la longitud en caracteres y por ultimo los convertimos a mayusculas.

En lugar de usar un array vamos a usar una List<string> (el funcionamiento es similar)

In [ ]:
List<string> nombres = new() {
  "Tom", "Dick", "Harry", "Mary", "Jay"
};

var resultado = nombres
                  .Where(n => n.Contains('a'))
                  .OrderBy(p => p.Length)
                  .Select(x => x.ToUpper());

foreach (var nom in resultado) Console.Write($"{nom} ");

##### Estructura de las consultas

En un DB engine las consultas se ejecutan en el orden inverso al que las escribimos, o sea, primero se resuelve el **from** luego el **where** y por ultimo el **select**. El order by es un caso particular ya que no forma parte de la constitucion del conjunto de datos que se obtiene sino mas bien es un agregado para la representacion visual.

Recordemos que un conjunto (desde el punto de vista relacional) no tiene un orden establecido, la ubicacion de cada elemento o fila en el set es irrelevante.

En LINQ veremos que el **order by** se puede procesar antes o despues del select. Aca es donde vamos a tener que pensar un poco mas y preguntarnos...

<div 
  style="position: relative; padding: 1rem 1rem; margin-bottom: 1rem;
         border: 1px solid transparent; border-radius: 0.25rem; color: black;
         background-color: #ffd966; border-color: black; width: 90%">
  Tiene sentido hacer un order by antes del select, si despues quizas el select nos hace perder el ordenamiento previo?
</div>

Veamos un ejemplo, usando un tipo **record** para simular una entidad de nuestro modelo

In [ ]:
public record Persona(string Nombre, int Edad, DateTime Nacimiento);

Persona[] nombres = new[] {
  new Persona("Persona 1", 30, DateTime.Now),
  new Persona("Person 2", 50, DateTime.MaxValue),
  new Persona("Persona 3", 100, DateTime.MinValue)
};
  
var xx = nombres
            .Where(p => p.Nombre.Contains("a"))
            .OrderBy(p => p.Nombre)
            .Select(p => new {p.Edad, p.Nacimiento}) ;

xx.Display();

xx = nombres
      .Where(p => p.Nombre.Contains("a"))
      .Select(p => new {p.Edad, p.Nacimiento})
      .OrderBy(anonimo => anonimo.Nacimiento) ;

xx.Display();

Entonces, tiene sentido ordenar por Nombre si despues perdemos este atributo?

O tiene mas sentido aplicar el OrderBy al final como hace cualquier dialecto de SQL y poder ordenar sobre los campos proyectados (los cuales a su vez podrian ser calculados!)

Ambas posibilidades estan disponibles en fluent, sin embargo en query siempre debemos colocar al final una clausula select!

##### Uso de into en query syntax para continuar la consulta

Cuando llegamos a la clausula select, por ejemplo para crear un objeto de tipo anonimo como vimos en los ejemplo previos, pero necesitamos continuar con la consulta, en query syntax no podemos simplemente seguir con un where ya que select es la clausula final.

Tenemos que usar entonces una palabra clave especial **into** que tiene como argumento un identificador. Este identificador sera como una segunda variable de secuencia que podemos empezar a usar desde ese momento. En el ejemplo teniamo a *nom* en el inicio y luego tendremos a *anonimo* pero como podemos entender, anonimo ahora tendra los dos campos que nos quedaron luego del select inicial.

Con esta segunda variable podemos repetir la clausula where y un nuevo select.

Abajo esta la misma sentencia pero expresada como fluent. Como en este caso no tenemos una variable de secuencia que se mantiene en todas las clausulas (las expresiones lambda pueden tener cualquier nombre de argumento) no necesitamos tener un equivalente a *into* puesto el primer select ya retorna un `IEnumerable<tipo anonimo>` y no tenemos que esforzarnos por generar una nueva variable de secuencia.

Tambien vemos que al no ser obligatorio terminar la consulta con Select() tambien nos estamos ahorrando este paso.

In [ ]:
public record Persona(string Nombre, int Edad, DateTime Nacimiento);

Persona[] nombres = new[] {
  new Persona("Persona 1", 30, DateTime.Now),
  new Persona("Person 2", 50, DateTime.MaxValue),
  new Persona("Persona 3", 100, DateTime.MinValue)
};
  
var xx = from nom in nombres
            where nom.Nombre.Contains("a")
            select new { nom.Edad, nom.Nacimiento }
              into anonimo
              where anonimo.Edad == 30
              select anonimo;

xx.Display();

var yy = nombres
            .Where(x => x.Nombre.Contains('a'))
            .Select(p=> new {p.Edad, p.Nacimiento})
            .Where(an => an.Edad == 30);

yy.Display();

##### Como retornar tipos anonimos

Los tipos anonimos no pueden usarse como tipo de retporno en una funcion o metodo. Esto es trivial ya que no podemos nombrar un tipo anonimo, el compilador lo infiere analizando las diferentes llamadas a metodos de extension, pero no podemos nunca usarlo como resultado de llamada a un metodo.

Para retornar un resultado generado en un metodo Select() tendremos que usar un tipo bien conocido, una class/struct/record o bien una tupla.

Vamos a repetir el caso anterior devolviendo primero un record y luego una tupla. Eliminamos el ultimo Where para que por lo menos tengamos dos elementos.

Creamos el record con nombres diferentes para que veamos que la construccion del record es posicional.

In [ ]:
//  retornamos un IEnumerable del tipo anonimo ad-hoc
//
public record Persona(string Nombre, int Edad, DateTime Nacimiento);

public record TipoAnonimo(int EdadPersona, DateTime FechaNacimiento);

GetPersonas().Display();

IEnumerable<TipoAnonimo> GetPersonas()
{
  Persona[] nombres = new[] {
    new Persona("Persona 1", 30, DateTime.Now),
    new Persona("Person 2", 50, DateTime.MaxValue),
    new Persona("Persona 3", 100, DateTime.MinValue)
  };

  var yy = nombres
              .Where(x => x.Nombre.Contains('a'))
              .Select(p=> new TipoAnonimo(p.Edad, p.Nacimiento));

  return yy;
}

En el caso de la tupla, vemos que directamente retornamos un "literal" de tupla

En la declaracion del metodo, el tipo es IEnumerable de la tupla y en este caso podemos renombrar los nombres de cada item de la tupla.

In [ ]:
//  retornamos un IEnumerable de una tupla creada para este fin
//
public record Persona(string Nombre, int Edad, DateTime Nacimiento);


foreach (var x in GetPersonas()) Console.WriteLine($"Años: {x.Años} ; Cumple: {x.Cumple}");

IEnumerable<(int Años, DateTime Cumple)> GetPersonas()
{
  Persona[] nombres = new[] {
    new Persona("Persona 1", 30, DateTime.Now),
    new Persona("Person 2", 50, DateTime.MaxValue),
    new Persona("Persona 3", 100, DateTime.MinValue)
  };

  var yy = nombres
              .Where(x => x.Nombre.Contains('a'))
              .Select(p => (p.Edad, p.Nacimiento));

  return yy;
}